# AIS project

### Import libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from geopy.distance import geodesic
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import xgboost as xgb

import geopy.distance

### Load data from CSV

In [6]:
# Load data
ais_train_import = pd.read_csv('data/ais_train.csv', sep='|')  # Adjust file path
ais_eval_import = pd.read_csv('data/ais_test.csv', sep=',')  # Adjust file path

vessels_import = pd.read_csv('data/vessels.csv', sep='|')  # Adjust file path
ports_import = pd.read_csv('data/ports.csv', sep='|')  # Adjust file path
schedules_import = pd.read_csv('data/schedules_to_may_2024.csv', sep='|')  # Adjust file path
# Convert latitude and longitude to float32
ais_train_import['latitude'] = ais_train_import['latitude'].astype('float32')
ais_train_import['longitude'] = ais_train_import['longitude'].astype('float32')
ports_import['latitude'] = ports_import['latitude'].astype('float32')
ports_import['longitude'] = ports_import['longitude'].astype('float32')
schedules_import['portLatitude'] = schedules_import['portLatitude'].astype('float32')
schedules_import['portLongitude'] = schedules_import['portLongitude'].astype('float32')

### Preprocess data

In [7]:
# Preprocess data

# Make copies of the loaded datasets
# time|cog|sog|rot|heading|navstat|etaRaw|latitude|longitude|vesselId|portId
train = ais_train_import[['vesselId', 'time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude', 'longitude', 'portId']]
eval = ais_eval_import[['ID', 'vesselId', 'time']].copy()
vessels = vessels_import[['vesselId', 'shippingLineId']].copy()
ports = ports_import[['portId', 'longitude', 'latitude']].copy()
schedules = schedules_import[['vesselId', 'shippingLineId', 'arrivalDate', 'sailingDate', 'portId', 'portLatitude', 'portLongitude']].copy()

ports = ports.rename(columns={'longitude': 'portLongitude', 'latitude': 'portLatitude'})

# Dropp missing schedules
schedules = schedules.dropna()

# Connect data to AIS data
train = pd.merge(train, vessels, on='vesselId', how='left')
train = pd.merge(train, ports, on='portId', how='left')

eval = pd.merge(eval, vessels, on='vesselId', how='left')


# Convert time to datetime
train['time'] = pd.to_datetime(train['time'])
eval['time'] = pd.to_datetime(eval['time'])

# Label encode vesselId
le_vesselId = LabelEncoder()
le_vesselId.fit(train['vesselId'])
train['vesselId'] = le_vesselId.transform(train['vesselId'])
eval['vesselId'] = le_vesselId.transform(eval['vesselId'])

# Label encode portId
le_portId = LabelEncoder()
le_portId.fit(pd.concat([train['portId'], schedules['portId']]))
train['portId'] = le_portId.transform(train['portId'])
schedules['portId'] = le_portId.transform(schedules['portId'])
#eval['portId'] = le_portId.transform(eval['portId'])

display(train)
display(eval)


,vesselId,time,cog,sog,rot,heading,navstat,latitude,longitude,portId,shippingLineId,portLongitude,portLatitude
0,50,2024-01-01 00:00:25,284.0,0.7,0,88,0,-34.743698,-57.851299,40,61ec65aea8cafc0e93f0e900,-71.618889,-33.587502
1,189,2024-01-01 00:00:36,109.6,0.0,-6,347,1,8.894400,-79.479393,682,61be24564ea00ae59d0fe37a,-79.532997,8.967000
2,432,2024-01-01 00:01:45,111.0,11.0,0,112,0,39.190651,-76.475670,358,61be24564ea00ae59d0fe379,-76.558891,39.232498
3,110,2024-01-01 00:03:11,96.4,0.0,0,142,1,-34.411888,151.020676,18,61a8e672f9cba188601e84ac,150.899445,-34.462502
4,356,2024-01-01 00:03:51,214.0,19.7,0,215,0,35.883789,-5.916360,613,61be24564ea00ae59d0fe37a,-5.817000,35.783001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522060,682,2024-05-07 23:59:07,359.1,13.4,0,1,0,52.191311,-5.822230,568,61a8e673f9cba188601e84b3,-7.100000,52.250000
1522061,85,2024-05-07 23:59:08,12.3,17.1,0,13,0,38.961418,-12.005020,686,61a8e672f9cba188601e84ac,-9.417000,38.700001
1522062,459,2024-05-07 23:59:08,269.8,14.9,-1,270,0,49.713718,-5.220420,759,61a8e673f9cba188601e84b9,-5.317000,50.083000
1522063,596,2024-05-07 23:59:08,8.0,18.7,0,6,0,38.278950,10.782800,135,61ec6303a8cafc0e93f0e8f3,11.780833,42.098888


,ID,vesselId,time,shippingLineId
0,0,84,2024-05-08 00:03:16,61a8e672f9cba188601e84ac
1,1,623,2024-05-08 00:06:17,61be24574ea00ae59d0fe388
2,2,596,2024-05-08 00:10:02,61ec6303a8cafc0e93f0e8f3
3,3,542,2024-05-08 00:10:34,61be24564ea00ae59d0fe37a
4,4,1,2024-05-08 00:12:27,61ec94f1a8cafc0e93f0e92a
...,...,...,...,...
51734,51734,48,2024-05-12 23:59:58,61a8e672f9cba188601e84ab
51735,51735,110,2024-05-12 23:59:58,61a8e672f9cba188601e84ac
51736,51736,610,2024-05-12 23:59:58,61a8e673f9cba188601e84b9
51737,51737,574,2024-05-12 23:59:58,61ec643ca8cafc0e93f0e8f9


### Feature engineering

In [8]:
# Feature engineering

def time_features(df):
    df['hour'] = df['time'].dt.hour
    df['day'] = df['time'].dt.day
    df['dayofweek'] = df['time'].dt.dayofweek
    df['month'] = df['time'].dt.month

    # Cycle features
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['sin_day'] = np.sin(2 * np.pi * df['day'] / 30)
    df['cos_day'] = np.cos(2 * np.pi * df['day'] / 30)
    df['sin_dayofweek'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['cos_dayofweek'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)

    # drop hour, day, dayofweek, month
    df = df.drop(['hour', 'day', 'dayofweek', 'month', 'time'], axis=1)

    return df

train = time_features(train)
eval = time_features(eval)

# ShippinglineID to label encoding
le_shippingLineId = LabelEncoder()
combined_shippingline_ids = pd.concat([train['shippingLineId'], schedules['shippingLineId']]).unique()
le_shippingLineId.fit(combined_shippingline_ids)
train['shippingLineId'] = le_shippingLineId.transform(train['shippingLineId'])
eval['shippingLineId'] = le_shippingLineId.transform(eval['shippingLineId'])
schedules['shippingLineId'] = le_shippingLineId.transform(schedules['shippingLineId'])

# Use information from schedules to check which shipping lines goes to what ports
shippingline_ports = schedules[['shippingLineId', 'portId', 'portLongitude', 'portLatitude']].drop_duplicates()


# Create a dictionary mapping shippingLineId to possible port coordinates
shippingline_ports_dict = shippingline_ports.groupby('shippingLineId')[['portLongitude', 'portLatitude']].apply(lambda x: x.values.tolist()).to_dict()

display(shippingline_ports)
display(shippingline_ports_dict)
display(train.head())
display(eval.head())

,shippingLineId,portId,portLongitude,portLatitude
0,1,379,-81.496666,31.140556
1,1,351,-1.428889,50.902500
2,1,98,8.554722,53.563610
3,1,366,-74.028610,40.688332
4,1,270,-79.879791,9.372370
...,...,...,...,...
72552,5,136,8.916389,44.404720
87936,0,71,-69.881111,18.475279
106065,0,242,-9.635833,30.423889
117119,1,409,-56.206669,-34.900555


{0: [[31.0272216796875, -29.88111114501953],
  [56.74055480957031, 24.378332138061523],
  [48.15972137451172, 29.044721603393555],
  [-81.56444549560547, 30.38083267211914],
  [25.029460906982422, 51.6245002746582],
  [-76.8294448852539, 17.981666564941406],
  [-79.92749786376953, 32.82222366333008],
  [72.8852767944336, 18.941944122314453],
  [-93.96111297607422, 29.843055725097656],
  [-94.08333587646484, 30.07805633544922],
  [-95.31916809082031, 28.9505558013916],
  [-96.1338882446289, 19.208332061767578],
  [121.18555450439453, 31.65833282470703],
  [4.828332901000977, 52.413055419921875],
  [33.64277648925781, 34.92416763305664],
  [23.61039924621582, 37.951141357421875],
  [3.2072219848632812, 51.336387634277344],
  [-1.4288890361785889, 50.90250015258789],
  [151.7705535888672, -32.907501220703125],
  [4.299722194671631, 51.29777908325195],
  [14.511943817138672, 35.89527893066406],
  [-3.8077778816223145, 43.442222595214844],
  [150.89944458007812, -34.462501525878906],
  [144

,vesselId,cog,sog,rot,heading,navstat,latitude,longitude,portId,shippingLineId,portLongitude,portLatitude,sin_hour,cos_hour,sin_day,cos_day,sin_dayofweek,cos_dayofweek,sin_month,cos_month
0,50,284.0,0.7,0,88,0,-34.743698,-57.851299,40,23,-71.618889,-33.587502,0.0,1.0,0.207912,0.978148,0.0,1.0,0.5,0.866025
1,189,109.6,0.0,-6,347,1,8.894400,-79.479393,682,15,-79.532997,8.967000,0.0,1.0,0.207912,0.978148,0.0,1.0,0.5,0.866025
2,432,111.0,11.0,0,112,0,39.190651,-76.475670,358,14,-76.558891,39.232498,0.0,1.0,0.207912,0.978148,0.0,1.0,0.5,0.866025
3,110,96.4,0.0,0,142,1,-34.411888,151.020676,18,1,150.899445,-34.462502,0.0,1.0,0.207912,0.978148,0.0,1.0,0.5,0.866025
4,356,214.0,19.7,0,215,0,35.883789,-5.916360,613,15,-5.817000,35.783001,0.0,1.0,0.207912,0.978148,0.0,1.0,0.5,0.866025


,ID,vesselId,shippingLineId,sin_hour,cos_hour,sin_day,cos_day,sin_dayofweek,cos_dayofweek,sin_month,cos_month
0,0,84,1,0.0,1.0,0.994522,-0.104528,0.974928,-0.222521,0.5,-0.866025
1,1,623,18,0.0,1.0,0.994522,-0.104528,0.974928,-0.222521,0.5,-0.866025
2,2,596,21,0.0,1.0,0.994522,-0.104528,0.974928,-0.222521,0.5,-0.866025
3,3,542,15,0.0,1.0,0.994522,-0.104528,0.974928,-0.222521,0.5,-0.866025
4,4,1,25,0.0,1.0,0.994522,-0.104528,0.974928,-0.222521,0.5,-0.866025


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, accuracy_score

# Define features and target
features_classifier = ['vesselId', 'sin_hour', 'cos_hour', 'sin_day', 'cos_day', 'sin_dayofweek', 'cos_dayofweek', 'sin_month', 'cos_month', 'shippingLineId']
target_classifier = ['portLongitude', 'portLatitude']

input_tts = train[features_classifier+target_classifier].dropna()

# Prepare training and validation data
X_train_port, X_val_port, y_train_port, y_val_port = train_test_split(input_tts[features_classifier], input_tts[target_classifier], random_state=42, test_size=0.2)

model_ports = MultiOutputRegressor(RandomForestRegressor())



In [ ]:
model_ports.fit(X_train_port,y_train_port)

In [11]:
# Save model
import pickle
model_ports_csv = 'models/model_ports.sav'
pickle.dump(model_ports, open(model_ports_csv, 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'models/model_ports.sav'

In [12]:
# Function to find the closest valid port coordinates
def find_closest_port(predicted_coords, valid_ports):
    closest_port = min(valid_ports, key=lambda port: np.linalg.norm(np.array(port) - np.array(predicted_coords)))
    return closest_port

# Prediction function with filter
def predict_with_filter(model, data, shippingline_ports_dict):
    predictions = []
    for index, row in data.iterrows():
        shipping_line_id = row['shippingLineId']
        valid_ports = shippingline_ports_dict.get(shipping_line_id)
        
        # Get model predictions
        model_predictions = model.predict([row])[0]
        
        # Find the closest valid port coordinates
        closest_port = find_closest_port(model_predictions, valid_ports)
        
        predictions.append(closest_port)
    
    return predictions

# Example usage
predictions_port = predict_with_filter(model_ports, X_val_port, shippingline_ports_dict)

# Evaluate the model
mse = mean_squared_error(y_val_port, predictions_port)
print("Mean Squared Error:", mse)

NotFittedError: This MultiOutputRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### Train test split

In [13]:
from sklearn.model_selection import train_test_split

# Function to split data for each vessel
def split_vessel_data(df, vessel_id_col='vesselId', test_size=0.2):
    train_list = []
    test_list = []
    
    for vessel_id in df[vessel_id_col].unique():
        vessel_data = df[df[vessel_id_col] == vessel_id]
        if len(vessel_data) > 1:  # Ensure there are enough samples to split
            train_data, test_data = train_test_split(vessel_data, test_size=test_size, random_state=False)
            train_list.append(train_data)
            test_list.append(test_data)
        else:
            train_list.append(vessel_data)  # If not enough samples, add all to train_list
    
    train_combined = pd.concat(train_list).reset_index(drop=True)
    test_combined = pd.concat(test_list).reset_index(drop=True)
    
    return train_combined, test_combined

# Split the data
train_split, test_split = split_vessel_data(train)

X_train = train_split[features]
y_train = train_split[target]

X_val = test_split[features]
y_val = test_split[target]

# Display the results
print('X_train:', X_train.shape)
display(X_train.head())
print('y_train:', y_train.shape)
display(y_train.head())
print('X_val:', X_val.shape)
display(X_val.head())
print('y_val:', y_val.shape)
display(y_val.head())

NameError: name 'features' is not defined

### Prep data for LSTM (scale and converd to 3D-array)

In [14]:


# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)


timesteps = 10
# Reshape the data for LSTM (samples, timesteps, features)
# Assuming we are creating sequences of 'timesteps' for each sample
def create_sequences(X, timesteps):
    sequences = []
    for i in range(len(X) - timesteps):
        sequences.append(X[i:i + timesteps])
    return np.array(sequences)

X_train_seq = create_sequences(X_train, timesteps)
X_val_seq = create_sequences(X_val, timesteps)

# Reshape your target variables (y_train, y_val) in the same way
y_train_seq = create_sequences(y_train, timesteps)
y_val_seq = create_sequences(y_val, timesteps)

# Check the final shapes
print(X_train_seq.shape)  # Should be (number of samples, timesteps, number of features)
print(y_train_seq.shape)  # Should be (number of samples, timesteps, 2) for lat/lon


NameError: name 'X_train' is not defined

### Define, compile and train model

In [4]:
# LSTM model

def check_for_nans(generator):
    for batch in generator:
        if np.any(np.isnan(batch[0])) or np.any(np.isnan(batch[1])):
            print("Found NaNs in the data!")
            break

# Input shape
n_features = X_train_seq.shape[2]

print(n_features)

inputs = Input(shape=(None, n_features))

# LSTM layers
x = LSTM(units=64, return_sequences=True)(inputs)
x = LSTM(units=32)(x)

# Latitude and Longitude (continuous outputs)
lat_output = Dense(units=1, name='latitude')(x)
lon_output = Dense(units=1, name='longitude')(x)

# Define model
model_LSTM = Model(inputs=inputs, outputs=[lat_output, lon_output])

# Compile the model
model_LSTM.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'latitude': 'mse',
        'longitude': 'mse'
    },
    metrics={
        'latitude': 'mae',
        'longitude': 'mae'
    },
    loss_weights={'latitude': 1.0, 'longitude': 1.0}
)


# Reshape target variables to match the expected shape
y_train_seq_lat = y_train_seq[:, :, 0].reshape(-1, timesteps, 1)
y_train_seq_lon = y_train_seq[:, :, 1].reshape(-1, timesteps, 1)
y_val_seq_lat = y_val_seq[:, :, 0].reshape(-1, timesteps, 1)
y_val_seq_lon = y_val_seq[:, :, 1].reshape(-1, timesteps, 1)

model_LSTM.summary()


NameError: name 'X_train_seq' is not defined

In [14]:

# Fit the model
model_LSTM.fit(
    x=X_train_seq,
    y={'latitude': y_train_seq_lat, 'longitude': y_train_seq_lon},
    epochs=50,
    batch_size=128,
    validation_data=(X_val_seq, {'latitude': y_val_seq_lat, 'longitude': y_val_seq_lon})
)   


14


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 14)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, None, 64)  │     20,224 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │     12,416 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latitude (Dense)    │ (None, 1)         │         33 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ longitude (Dense)   │ (None, 1)         │         33 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,706 (127.76 KB)

 Trainable params: 32,706 (127.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
9501/9501 ━━━━━━━━━━━━━━━━━━━━ 230s 23ms/step - latitude_loss: 0.5907 - latitude_mse: 0.5907 - longitude_loss: 0.6908 - longitude_mse: 0.6908 - loss: 1.2816 - val_latitude_loss: 0.4903 - val_latitude_mse: 0.4903 - val_longitude_loss: 0.5635 - val_longitude_mse: 0.5634 - val_loss: 1.0538
Epoch 2/50
9501/9501 ━━━━━━━━━━━━━━━━━━━━ 198s 21ms/step - latitude_loss: 0.4721 - latitude_mse: 0.4721 - longitude_loss: 0.5516 - longitude_mse: 0.5516 - loss: 1.0237 - val_latitude_loss: 0.4783 - val_latitude_mse: 0.4783 - val_longitude_loss: 0.5480 - val_longitude_mse: 0.5480 - val_loss: 1.0262
Epoch 3/50
9501/9501 ━━━━━━━━━━━━━━━━━━━━ 92s 10ms/step - latitude_loss: 0.4560 - latitude_mse: 0.4560 - longitude_loss: 0.5344 - longitude_mse: 0.5344 - loss: 0.9904 - val_latitude_loss: 0.4639 - val_latitude_mse: 0.4639 - val_longitude_loss: 0.5351 - val_longitude_mse: 0.5351 - val_loss: 0.9990
Epoch 4/50
9501/9501 ━━━━━━━━━━━━━━━━━━━━ 155s 16ms/step - latitude_loss: 0.4489 - latitude_mse: 0.4489 

### Save model

In [3]:
# Save the model
model_LSTM.save('lstm_model_2.keras')

NameError: name 'model_LSTM' is not defined

### Evaluate model

In [2]:
# Evaluate the model on validation data
val_loss, val_lat_mae, val_lon_mae = model_LSTM.evaluate(X_val_seq, {'latitude': y_val_seq_lat, 'longitude': y_val_seq_lon})

print(f"Validation Loss: {val_loss}")
print(f"Validation Latitude MAE: {val_lat_mae}")
print(f"Validation Longitude MAE: {val_lon_mae}")

# Make predictions on the validation set
y_val_pred = model_LSTM.predict(X_val_seq)
y_val_pred_lat = y_val_pred[0].reshape(-1, 1)
y_val_pred_lon = y_val_pred[1].reshape(-1, 1)

# Inverse transform the predictions
y_val_pred = np.concatenate([y_val_pred_lat, y_val_pred_lon], axis=1)
y_val_pred = scaler_y.inverse_transform(y_val_pred)

# Calculate evaluation metrics
mse = mean_squared_error(y_val, y_val_pred)
mae = np.mean(np.abs(y_val - y_val_pred), axis=0)

print(f"Validation Mean Squared Error: {mse}")
print(f"Validation Mean Absolute Error: {mae}")

NameError: name 'model_LSTM' is not defined

### Predict the future

In [1]:
# predict on eval data
X_eval = eval[features]
X_eval = scaler.transform(X_eval)  # Ensure the correct columns are selected
X_eval = create_sequences(X_eval, timesteps)

# import the model from keras
from tensorflow.keras.models import load_model

# Load the model
model_LSTM = load_model('lstm_model_2.keras')

# Make predictions
y_eval_pred = model_LSTM.predict(X_eval)
y_eval_pred_lat = y_eval_pred[0].reshape(-1, 1)
y_eval_pred_lon = y_eval_pred[1].reshape(-1, 1)

display(y_eval_pred_lat)
display(y_eval_pred_lon)

# Inverse transform the predictions
y_eval_pred = np.concatenate([y_eval_pred_lat, y_eval_pred_lon], axis=1)
y_eval_pred = scaler_y.inverse_transform(y_eval_pred)

# Fill missing values with the first available prediction for both columns
first_value_fill = np.tile(y_eval_pred[0], (10, 1))  # Shape will be (10, 2)

# Concatenate the filled values with the actual predictions
predictions_filled = np.vstack([first_value_fill, y_eval_pred])


# Save the predictions
eval['latitude_predicted'] = predictions_filled[:, 0]
print(eval['latitude_predicted'])
eval['longitude_predicted'] = predictions_filled[:, 1]
eval[['ID', 'longitude_predicted', 'latitude_predicted']].to_csv('data/ais_eval_pred.csv', index=False)

NameError: name 'features' is not defined